In [1]:
import psycopg2
import pandas as pd


In [6]:


# Redshift connection parameters
host = 'redshift-cluster-1.c4u6tapvfqx0.us-east-2.redshift.amazonaws.com'
port = 5439
dbname = 'dev'
user = 'kbeleaws'
password = 'Kiranumass11'

# Establish connection
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password,
        connect_timeout=10 
    )
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")


Connection successful
